Video: 

https://www.youtube.com/watch?v=9ccl1_Wu24Q

Document: 

https://alejandro-ao.com/chat-with-mysql-using-python-and-langchain/

 - This is another example: 

    https://python.langchain.com/docs/use_cases/sql/quickstart/

Install LANGCHAIN

https://python.langchain.com/v0.1/docs/get_started/introduction

%pip install langchain mysql-connector-python 

%pip install langchain-openai

For OPEN AI, you need an APIKEY: https://platform.openai.com/

https://pypi.org/project/langchain-openai/


FREE SQLITE DATABASES https://www.sqlitetutorial.net/sqlite-sample-database/


In [3]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('..', 'secret')))
from langchain_community.utilities import SQLDatabase
database = '../data/chinook.db'
from secret_info import openai_key
import sqlite3
import pandas as pd
con = sqlite3.connect(database)

In [4]:
# SQL CHAIN
from langchain_core.prompts import ChatPromptTemplate
# The template is an string that ask for the question in this case: 
# schema is the schema from the database 
# question: the question that you want to answer
template = """Based on the table schema below, write a SQL query that would answer the user's question:
                {schema}

                Question: {question}
                SQL Query:"""
prompt = ChatPromptTemplate.from_template(template)

In [6]:
sqlite_uri = 'sqlite:///../data/chinook.db' 
db = SQLDatabase.from_uri(sqlite_uri)

def get_schema(_):
    # schema = db.get_table_info()
    return db.get_table_info()

my_schema = get_schema(db)
print(my_schema)


CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) NOT NULL, 
	"SupportRepId" INTEGER, 
	PRIMARY KEY ("CustomerId"), 
	FOREIGN KEY("SupportRepId") REFERENCES employe

In [7]:
result = prompt.format(schema = my_schema,  question = " How many users are there? ")
print(result)

Human: Based on the table schema below, write a SQL query that would answer the user's question:
                
CREATE TABLE albums (
	"AlbumId" INTEGER NOT NULL, 
	"Title" NVARCHAR(160) NOT NULL, 
	"ArtistId" INTEGER NOT NULL, 
	PRIMARY KEY ("AlbumId"), 
	FOREIGN KEY("ArtistId") REFERENCES artists ("ArtistId")
)

/*
3 rows from albums table:
AlbumId	Title	ArtistId
1	For Those About To Rock We Salute You	1
2	Balls to the Wall	2
3	Restless and Wild	2
*/


CREATE TABLE artists (
	"ArtistId" INTEGER NOT NULL, 
	"Name" NVARCHAR(120), 
	PRIMARY KEY ("ArtistId")
)

/*
3 rows from artists table:
ArtistId	Name
1	AC/DC
2	Accept
3	Aerosmith
*/


CREATE TABLE customers (
	"CustomerId" INTEGER NOT NULL, 
	"FirstName" NVARCHAR(40) NOT NULL, 
	"LastName" NVARCHAR(20) NOT NULL, 
	"Company" NVARCHAR(80), 
	"Address" NVARCHAR(70), 
	"City" NVARCHAR(40), 
	"State" NVARCHAR(40), 
	"Country" NVARCHAR(40), 
	"PostalCode" NVARCHAR(10), 
	"Phone" NVARCHAR(24), 
	"Fax" NVARCHAR(24), 
	"Email" NVARCHAR(60) N

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

# Create llm 
llm = ChatOpenAI(api_key = openai_key)
sql_chain = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm.bind(stop=["\nSQLResult:"])
    | StrOutputParser()
)

In [ ]:
user_question = 'how many albums are there in the database?'
sql_chain.invoke({"question": user_question})

In [ ]:
template = """Based on the table schema below, question, sql query, and sql response, write a natural language response:
{schema}

Question: {question}
SQL Query: {query}
SQL Response: {response}"""
prompt_response = ChatPromptTemplate.from_template(template)

def run_query(query):
    return db.run(query)



In [ ]:
full_chain = (
    RunnablePassthrough.assign(query=sql_chain).assign(
        schema=get_schema,
        response=lambda vars: run_query(vars["query"]),
    )
    | prompt_response
    | model
)


In [ ]:
user_question = 'how many albums are there in the database?'
full_chain.invoke({"question": user_question})

# 'There are 347 albums in the database.'


STILL MORE IN THE VIDEO
_____

_____

# Anthropic - NOT A FREE PLAN

Documentation

https://python.langchain.com/docs/integrations/chat/anthropic/

anthropic api key: 

https://console.anthropic.com/settings/keys


Install: 

%pip install -qU langchain-anthropic



In [11]:
from secret_info import anthropic_key
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate

chat = ChatAnthropic(temperature=0, api_key=anthropic_key, model_name="claude-3-opus-20240229")

system = (
    "You are a helpful assistant that translates {input_language} to {output_language}."
)
human = "{text}"
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", human)])

chain = prompt | chat
chain.invoke(
    {
        "input_language": "English",
        "output_language": "Korean",
        "text": "I love Python",
    }
)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'Your credit balance is too low to access the Claude API. Please go to Plans & Billing to upgrade or purchase credits.'}}

_____ 

# OCTO AI

In [ ]:
from secret_info import octo

from langchain_community.chat_models import ChatOctoAI
from langchain_core.messages import HumanMessage, SystemMessage


In [ ]:
chat = ChatOctoAI(max_tokens=300, octoai_api_token = octo, model_name="mixtral-8x7b-instruct")



In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Tell me about Leonardo da Vinci briefly."),
]
print(chat(messages).content)